In [1]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Import LendClub csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('/content/drive/MyDrive/24_1/Lending Club/lending_club_2020_train.csv')
df.head()

<ipython-input-2-96ac412dc8b2>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/24_1/Lending Club/lending_club_2020_train.csv')


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,124989905,6000.0,6000.0,6000.0,36 months,7.97%,187.94,A,A5,Teacher,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,139665484,35000.0,35000.0,35000.0,60 months,14.47%,822.95,C,C2,Product Specialist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,84253847,23200.0,23200.0,23200.0,60 months,24.99%,680.82,E,E4,Production Foreman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,126296074,4500.0,4500.0,4500.0,36 months,9.93%,145.06,B,B2,Office Manager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,148913312,15000.0,15000.0,15000.0,36 months,16.40%,530.33,C,C4,Secretary,...,May-2020,Jul-2020,May-2020,2.0,0.0,ACTIVE,413.83,11025.25,54.62,N


In [3]:
# Convert 'issue_d' to datetime objects
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')

# Filter by specified date range
df = df[(df['issue_d'] >= '2013-01-01') & (df['issue_d'] <= '2014-12-31')]

In [4]:
# Map 'loan_status' values to 0 and 1
df['loan_status_encoded'] = df['loan_status'].apply(lambda x: 0 if x in ['Fully Paid'] else 1)

In [6]:
# Get rid of post-loan variables

columns_to_drop = [
    'collection_recovery_fee',
    'collections_12_mths_ex_med',
    'deferral_term',
    'funded_amnt',
    'funded_amnt_inv',
    'hardship_amount',
    'hardship_dpd',
    'hardship_end_date',
    'hardship_flag',
    'hardship_last_payment_amount',
    'hardship_length',
    'hardship_loan_status',
    'hardship_payoff_balance_amount',
    'hardship_reason',
    'hardship_start_date',
    'hardship_status',
    'hardship_type',
    'last_fico_range_high',
    'last_fico_range_low',
    'last_pymnt_amnt',
    'last_pymnt_d',
    'mths_since_last_major_derog',
    'next_pymnt_d',
    'orig_projected_additional_accrued_interest',
    'out_prncp',
    'out_prncp_inv',
    'payment_plan_start_date',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_int',
    'total_rec_late_fee',
    'total_rec_prncp',
    'recoveries',

    ]  # Add other column names to this list

df = df.drop(columns=columns_to_drop)

KeyError: "['collection_recovery_fee', 'collections_12_mths_ex_med', 'deferral_term', 'funded_amnt', 'funded_amnt_inv', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 'hardship_flag', 'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'mths_since_last_major_derog', 'next_pymnt_d', 'orig_projected_additional_accrued_interest', 'out_prncp', 'out_prncp_inv', 'payment_plan_start_date', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'recoveries'] not found in axis"

In [7]:
#Get rid of features with 99% null values

threshold = 0.99

# Calculate the proportion of null values for each column
null_proportions = df.isnull().mean()

# Get the names of columns to drop
columns_to_drop = null_proportions[null_proportions >= threshold].index.tolist()

# Drop the columns
df = df.drop(columns=columns_to_drop)

In [8]:
#Get rid of features with the same values

threshold = 0.99

# Calculate the proportion of the most frequent value for each column
dominant_proportions = df.apply(lambda x: x.value_counts(normalize=True).iloc[0] if not x.value_counts().empty else 0)

# Get the names of columns to drop
columns_to_drop = dominant_proportions[dominant_proportions >= threshold].index.tolist()

# Drop the columns
df = df.drop(columns=columns_to_drop)

In [9]:
remaining_columns = df.columns.tolist()
remaining_columns.sort()  # Sort the list alphabetically
num_columns = len(remaining_columns)

print("Remaining columns (alphabetical order):", remaining_columns)
print("Number of remaining columns:", num_columns)

Remaining columns (alphabetical order): ['acc_open_past_24mths', 'addr_state', 'annual_inc', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'debt_settlement_flag', 'delinq_2yrs', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade', 'home_ownership', 'id', 'initial_list_status', 'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'last_credit_pull_d', 'loan_amnt', 'loan_status', 'loan_status_encoded', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_last_delinq', 'mths_since_last_record', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec', 'pub_re

In [10]:
#Emp_length convert to float

def map_emp_length(value):
  if pd.isna(value):
    return value
  elif value == '< 1 year':
    return 0
  elif value == '10+ years':
    return 10
  else:
    return int(value.split()[0])  # Extract numeric value from strings like '2 years'

df['emp_length'] = df['emp_length'].apply(map_emp_length)

In [11]:
#earliest_cr_line convert to float

from datetime import datetime

# Convert 'earliest_cr_line' to datetime objects
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')

# Calculate years from September 2018
reference_date = datetime(2018, 9, 1)
df['earliest_cr_line'] = (reference_date - df['earliest_cr_line']).dt.days / 365.25

In [15]:
# Remove '%' sign and convert to numeric
df['revol_util'] = pd.to_numeric(df['revol_util'].str.rstrip('%'), errors='coerce')

# Divide by 100 to convert percentages to decimals
df['revol_util'] = df['revol_util'] / 100

AttributeError: Can only use .str accessor with string values!

In [17]:
# Remove '%' sign and convert to numeric
df['int_rate'] = pd.to_numeric(df['int_rate'].str.rstrip('%'), errors='coerce')

# Divide by 100 to convert percentages to decimals
df['int_rate'] = df['int_rate'] / 100

AttributeError: Can only use .str accessor with string values!

In [18]:
#Fill null values with 0 for 'emp_length'

df['emp_length'] = df['emp_length'].fillna(0)

In [21]:
#One hot encoding for map_home_ownership
def map_home_ownership(value):
  if value in ['MORTGAGE', 'OWN']:
    return 0
  else:
    return 1

df['home_ownership'] = df['home_ownership'].apply(map_home_ownership)

In [22]:
#One hot encoding for map_term
def map_term(value):
  if value == ' 36 months':
    return 0
  else:
    return 1

df['term'] = df['term'].apply(map_term)

In [23]:
#One hot encoding for map_verification_status
def map_verification_status(value):
  if value in ['Verified', 'Source Verified']:
    return 0
  else:
    return 1

df['verification_status'] = df['verification_status'].apply(map_verification_status)

In [24]:
#Fill null values with -1
columns_to_fill = [
    'mths_since_last_delinq',
    'mths_since_last_record',
    'mths_since_recent_bc',
    'mths_since_recent_bc_dlq',
    'mths_since_recent_inq',
    'mths_since_recent_revol_delinq',
    'delinq_2yrs'

    ]  # Add other column names to this list

df[columns_to_fill] = df[columns_to_fill].fillna(-1)

In [26]:
# How many null values for float columns
float_columns = df.select_dtypes(include=['float']).columns
null_percentages = df[float_columns].isnull().mean() * 100
print(null_percentages)

loan_amnt                         0.000000
int_rate                          0.000000
installment                       0.000000
emp_length                        0.000000
annual_inc                        0.000000
dti                               0.000000
delinq_2yrs                       0.000000
earliest_cr_line                  0.000000
fico_range_low                    0.000000
fico_range_high                   0.000000
inq_last_6mths                    0.000000
mths_since_last_delinq            0.000000
mths_since_last_record            0.000000
open_acc                          0.000000
pub_rec                           0.000000
revol_bal                         0.000000
revol_util                        0.059002
total_acc                         0.000000
tot_coll_amt                      0.000000
tot_cur_bal                       0.000000
total_rev_hi_lim                  0.000000
acc_open_past_24mths              0.000000
avg_cur_bal                       0.002702
bc_open_to_

In [27]:
#Fill null values with mean #1
columns_to_fill = [
    'mo_sin_old_il_acct',
    'mo_sin_old_rev_tl_op',
    'mo_sin_rcnt_rev_tl_op',
    'mo_sin_rcnt_tl'
    ]

for column in columns_to_fill:
    df[column] = df[column].fillna(df[column].mean())

In [ ]:
#Fill null values with 0
columns_to_fill = ['pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies']

df[columns_to_fill] = df[columns_to_fill].fillna(0)

In [28]:
#Fill null values with mean for the rest of the float columns
float_columns = df.select_dtypes(include=['float']).columns

for column in float_columns:
    df[column] = df[column].fillna(df[column].mean())

In [29]:
# Object Columns to drop (not useful)

columns_to_drop = ['last_credit_pull_d', 'addr_state', 'grade', 'sub_grade', 'initial_list_status', 'issue_d', 'purpose', 'zip_code', 'loan_status']

df = df.drop(columns=columns_to_drop)

In [30]:
# Object Columns to drop (useful but don't know how)
columns_to_drop = ['debt_settlement_flag', 'emp_title', 'title', 'url']

df = df.drop(columns=columns_to_drop)

In [31]:
# Random Sampling to match defaults : not-default to 1:1
# Separate the DataFrame into two parts based on 'loan_status_encoded'
df_0 = df[df['loan_status_encoded'] == 0]
df_1 = df[df['loan_status_encoded'] == 1]

# Randomly sample 37545 rows from df_0
df_0_sampled = df_0.sample(n=37545, random_state=42)  # Set random_state for reproducibility

# Concatenate the sampled df_0 with df_1
df_balanced = pd.concat([df_0_sampled, df_1])

# Shuffle the balanced DataFrame
df_balanced = df_balanced.sample(frac=1, random_state=42)  # Shuffle rows for better model training

In [32]:
loan_status_counts = df_balanced['loan_status_encoded'].value_counts()
print("Counts:\n", loan_status_counts)

proportion = loan_status_counts[0] / loan_status_counts[1]
print("\nProportion of 0 to 1:", proportion)

Counts:
 loan_status_encoded
0    37545
1    37545
Name: count, dtype: int64

Proportion of 0 to 1: 1.0


In [38]:
#Export

#from google.colab import files

#df_balanced.to_csv('LC_Data_Cleaned_1.csv', index=False)
#files.download('LC_Data_Cleaned_1.csv')